In [129]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import SimpleRNN , LSTM , Dense , Activation , Dropout

In [130]:
df = pd.read_csv("all.csv")["content"]
df

0      Let the bird of loudest lay\r\nOn the sole Ara...
1      Sir Charles into my chamber coming in,\r\nWhen...
2      Our vice runs beyond all that old men saw,\r\n...
3      Lo I the man, whose Muse whilome did maske,\r\...
4      Long have I longd to see my love againe,\r\nSt...
                             ...                        
568    With the man I love who loves me not,\r\nI wal...
569    Hart Crane, "Voyages I, II, III, IV, V, VI" fr...
570    When you are old and grey and full of sleep,\r...
571    Give me hunger,\r\nO you gods that sit and giv...
572    Potuia, potuia\r\nWhite grave goddess,\r\nPity...
Name: content, Length: 573, dtype: object

In [131]:
text = df.str.cat(sep='\n').lower()
text
chars = sorted(list(set(text)))
chars

char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [132]:
window = 60
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - window, step):
    sentences.append(text[i: i + window]) 
    next_chars.append(text[i + window])
sentences = np.array(sentences)
next_chars = np.array(next_chars)

In [133]:
def getdata(sentences, next_chars):
    x = np.zeros((len(sentences), window, len(chars)), dtype=np.bool)
    y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
    length = len(sentences)
    index = 0
    for i in range(len(sentences)):
        sentence = sentences[i]
        for t, char in enumerate(sentence):
            x[i, t, char_indices[char]] = 1
        y[i, char_indices[next_chars[i]]] = 1
    return x , y

In [134]:
x , y = getdata(sentences , next_chars)

In [135]:
model = Sequential()

model.add( LSTM(64, input_shape=(window, len(chars))) )

model.add( Dense(len(chars)) )
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

In [136]:
model.fit(x , y , epochs=5 , batch_size=100)

Epoch 1/5
1893/1893 [==============================] - 157s 83ms/step - loss: 2.5950
Epoch 2/5
1893/1893 [==============================] - 159s 84ms/step - loss: 2.1944
Epoch 3/5
1893/1893 [==============================] - 151s 80ms/step - loss: 2.0901
Epoch 4/5
1893/1893 [==============================] - 154s 81ms/step - loss: 2.0210
Epoch 5/5
1893/1893 [==============================] - 185s 98ms/step - loss: 1.9719


In [137]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [138]:
generated = ''
original = sentence
sen = sentence

for i in range(400):
    xx = np.zeros((1, window, len(chars)))
    for t, char in enumerate(sen):
        xx[0, t, char_indices[char]] = 1.

    preds = model.predict(xx, verbose=0)[0]
    next_index = sample(preds)
    next_char = indices_char[next_index]

    generated += next_char
    sen = sen[1:] + next_char

print(original)
print(generated)

the sun
 the moon
shine the ojas 
oieeeeeiyimnioirrrrouoiio -nwiearoa:ilaldto
ael
siaerioearreiordsderii
